# Домашнее задание 2. Извлечение коллокаций.

При выполнении домашнего задания можно пользоваться тетрадками с семинаров.

### Описание задания:

1. Скачайте [корпус](https://github.com/sjut/HSE-Compling/blob/master/hw/testset2.txt) текстов, обработайте его с помощью UDPipe, извлеките все группы 
"глагол + прямое дополнение, выраженное существительным" 
(не учитывайте глаголы, которые встречаются в корпусе менее **50** раз).

2. Оцените полученные словосочетания следующими метриками: *log-likelihood*, *dice*, *PMI* (можно использовать `nltk.collocations`). 

3. Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: 
возьмите словосочетания, которые попадают в топ-100 по всем метрикам,
пересеките со [словарем глагольной сочетаемости](https://yadi.sk/d/5WWwOr9ccemcZA).

4. Добавьте в ЗС словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть), объясните свой выбор.

5. Оцените ранговую корреляцию (коэффициент Спирмена) результатов по каждой метрике с ЗС. 
Как это работает, читайте, например, [тут](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient#Example).
Можно использовать `scipy.stats.spearmanr`.
Опишите ошибки каждой метрики.

### Критерии оценки:

По 2 балла на каждый пункт.


In [1]:
import pandas as pd
from nltk.parse import DependencyGraph
from scipy.stats import spearmanr
import numpy as np
import nltk
from nltk.collocations import *
from collections import Counter
from itertools import chain

1. Cкачиваем корпус, обрабатываем его с помощью UDPipe, извлекаем все группы "глагол + прямое дополнение, выраженное существительным" (не учитываем глаголы, которые встречаются в корпусе менее 50 раз)

In [2]:
with open('testset2.txt', 'r', encoding = 'utf-8') as f:
    content = f.readlines()

In [3]:
!C:\Users\kapru\Desktop\HSE\NLP\4\HW\hw1\udpipe-1.2.0-bin\bin-win64\udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\Users\kapru\Desktop\HSE\NLP\4\HW\hw1\russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu

Loading UDPipe model: done.


In [3]:
trees = []

with open('testset2.conllu', encoding = 'utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [4]:
def lemma_triples(dep_graph, node=None):
    """
    Extract dependency triples of the form:
    ((head lemma, head tag), rel, (dep lemma, dep tag))
    """

    if not node:
        node = dep_graph.root

    if node['lemma']:
        head = (node['lemma'], node['ctag'])
    else:
        head = (node['word'], node['ctag'])
    for i in sorted(chain.from_iterable(node['deps'].values())):
        dep = dep_graph.get_by_address(i)
        if dep['lemma']:
            yield (head, dep['rel'], (dep['lemma'], dep['ctag']))
        else:
            yield (head, dep['rel'], (dep['word'], dep['ctag']))
        for triple in lemma_triples(dep_graph, node=dep):
            yield triple

In [9]:
direct_objects = []
for i, tree in enumerate(trees):
    try:
        g = DependencyGraph(tree, top_relation_label='root')
        for t in g.triples():
            if t[0][-1] == 'VERB' and t[1] == 'obj' and t[-1][-1] == 'NOUN':
                direct_objects.append(t)
    except Exception as e:
        print(i, e)
direct_objects[:10]

610 
1027 
1163 
1246 
1391 
1498 
1846 
2277 
2592 
2684 
2691 
2906 
5122 
5286 
5680 
5681 
5874 
6481 'NoneType' object is not subscriptable


[(('подал', 'VERB'), 'obj', ('иск', 'NOUN')),
 (('признать', 'VERB'), 'obj', ('договор', 'NOUN')),
 (('запускать', 'VERB'), 'obj', ('механизм', 'NOUN')),
 (('рассмотрит', 'VERB'), 'obj', ('вопрос', 'NOUN')),
 (('терпящих', 'VERB'), 'obj', ('бедствие', 'NOUN')),
 (('определят', 'VERB'), 'obj', ('срок', 'NOUN')),
 (('оберегали', 'VERB'), 'obj', ('покой', 'NOUN')),
 (('обвинили', 'VERB'), 'obj', ('руководство', 'NOUN')),
 (('вызвали', 'VERB'), 'obj', ('недовольство', 'NOUN')),
 (('привело', 'VERB'), 'obj', ('обитателей', 'NOUN'))]

In [7]:
direct_objects = []
for i, tree in enumerate(trees):
    try:
        g = DependencyGraph(tree, top_relation_label='root')
        for t in lemma_triples(g):
            if t[0][-1] == 'VERB' and t[1] == 'obj' and t[-1][-1] == 'NOUN':
                direct_objects.append(t)
    except Exception as e:
        print(i, e)

610 
1027 
1163 
1246 
1391 
1498 
1846 
2277 
2592 
2684 
2691 
2906 
5122 
5286 
5680 
5681 
5874 
6481 'NoneType' object is not subscriptable


In [8]:
direct_objects[:10]

[(('подать', 'VERB'), 'obj', ('иск', 'NOUN')),
 (('признать', 'VERB'), 'obj', ('договор', 'NOUN')),
 (('запускать', 'VERB'), 'obj', ('механизм', 'NOUN')),
 (('рассмотреть', 'VERB'), 'obj', ('вопрос', 'NOUN')),
 (('терпеть', 'VERB'), 'obj', ('бедствие', 'NOUN')),
 (('определить', 'VERB'), 'obj', ('срок', 'NOUN')),
 (('оберегать', 'VERB'), 'obj', ('покой', 'NOUN')),
 (('обвинить', 'VERB'), 'obj', ('руководство', 'NOUN')),
 (('вызвать', 'VERB'), 'obj', ('недовольство', 'NOUN')),
 (('привести', 'VERB'), 'obj', ('обитатель', 'NOUN'))]

In [9]:
verbs = []
for i, tree in enumerate(trees):
    try:
        g = DependencyGraph(tree, top_relation_label='root')
        for n in g.nodes:
            if g.nodes[n]['ctag'] == 'VERB':
                verbs.append(g.nodes[n]['lemma'])
    except Exception as e:
        print(i, e)

610 
1027 
1163 
1246 
1391 
1498 
1846 
2277 
2592 
2684 
2691 
2906 
5122 
5286 
5680 
5681 
5874 


In [11]:
verbs_cntr = Counter(verbs)

In [12]:
verbs = [v for v in verbs if verbs_cntr[v] >= 50]

In [13]:
len(verbs)

7246

In [14]:
direct_objects = [x for x in direct_objects if x[0][0] in verbs]

In [15]:
direct_objects = [(x[0][0], x[-1][0]) for x in direct_objects]

In [16]:
direct_objects[:10]

[('подать', 'иск'),
 ('признать', 'договор'),
 ('рассмотреть', 'вопрос'),
 ('обвинить', 'руководство'),
 ('получить', 'срок'),
 ('обвинить', 'олигарх'),
 ('направить', 'ведомство'),
 ('просить', 'суд'),
 ('делать', 'заявление'),
 ('рассматривать', 'ходатайство')]

In [14]:
tokens = []
for tree in trees:
    try:
        current_tokens = []
        g = DependencyGraph(tree, top_relation_label='root')
        for n in g.nodes:
            if g.nodes[n]['lemma']:
                current_tokens.append(g.nodes[n]['lemma'])
            else:
                current_tokens.append(g.nodes[n]['word'])
        tokens.append(current_tokens)
    except Exception as e:
        continue

2. Оцениваем полученные словосочетания следующими метриками:
 - log-likelihood
 - dice
 - PMI (можно использовать nltk.collocations).

In [19]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(tokens)

In [20]:
dice_scores = finder.score_ngrams(bigram_measures.dice)
pmi_scores = finder.score_ngrams(bigram_measures.pmi)
likelihood_scores = finder.score_ngrams(bigram_measures.likelihood_ratio)

In [24]:
do_dice_scores = {do: -100 for do in direct_objects}
do_pmi_scores = {do: -100 for do in direct_objects}
do_likelihood_scores = {do: -100 for do in direct_objects}
for do in direct_objects:
    dice_score = finder.score_ngram(bigram_measures.dice, do[0], do[-1])
    if dice_score:
        do_dice_scores[do] = dice_score
    pmi_score = finder.score_ngram(bigram_measures.pmi, do[0], do[-1])
    if pmi_score:
        do_pmi_scores[do] = pmi_score
    likelihood_score = finder.score_ngram(bigram_measures.likelihood_ratio, do[0], do[-1])
    if likelihood_score:
        do_likelihood_scores[do] = likelihood_score

In [25]:
top_100_dice = {k: v for k, v in sorted(do_dice_scores.items(), key=lambda item: item[1], reverse = True)[:100]}
top_100_pmi = {k: v for k, v in sorted(do_pmi_scores.items(), key=lambda item: item[1], reverse = True)[:100]}
top_100_likelik = {k: v for k, v in sorted(do_likelihood_scores.items(), key=lambda item: item[1], reverse = True)[:100]}

In [27]:
do_dice_scores

{('подать', 'иск'): 0.05282112845138055,
 ('признать', 'договор'): 0.0044943820224719105,
 ('рассмотреть', 'вопрос'): 0.03827751196172249,
 ('обвинить', 'руководство'): 0.011090573012939002,
 ('получить', 'срок'): 0.014760147601476014,
 ('обвинить', 'олигарх'): -100,
 ('направить', 'ведомство'): -100,
 ('просить', 'суд'): 0.008748481166464156,
 ('делать', 'заявление'): 0.017699115044247787,
 ('рассматривать', 'ходатайство'): -100,
 ('отказать', 'жалоба'): -100,
 ('делать', 'шаг'): -100,
 ('иметь', 'вид'): -100,
 ('оставить', 'заключение'): -100,
 ('дать', 'согласие'): 0.05,
 ('признать', 'сделка'): 0.0049261083743842365,
 ('обвинить', 'господин'): 0.0066815144766146995,
 ('удовлетворить', 'жалоба'): 0.03763440860215054,
 ('вынести', 'постановление'): 0.015873015873015872,
 ('получить', 'доля'): -100,
 ('вынести', 'приговор'): 0.0339943342776204,
 ('принять', 'решение'): 0.030245746691871456,
 ('признать', 'ответственность'): -100,
 ('выплатить', 'компенсация'): 0.0967741935483871,
 ('о

In [28]:
do_pmi_scores

{('подать', 'иск'): 4.584667336853158,
 ('признать', 'договор'): 2.255482165875268,
 ('рассмотреть', 'вопрос'): 5.959534584459041,
 ('обвинить', 'руководство'): 3.841929939590571,
 ('получить', 'срок'): 4.194363430477473,
 ('обвинить', 'олигарх'): -100,
 ('направить', 'ведомство'): -100,
 ('просить', 'суд'): 2.4043760311551274,
 ('делать', 'заявление'): 4.864017212862516,
 ('рассматривать', 'ходатайство'): -100,
 ('отказать', 'жалоба'): -100,
 ('делать', 'шаг'): -100,
 ('иметь', 'вид'): -100,
 ('оставить', 'заключение'): -100,
 ('дать', 'согласие'): 8.898761162187249,
 ('признать', 'сделка'): 2.9506275843468455,
 ('обвинить', 'господин'): 1.3158611279229824,
 ('удовлетворить', 'жалоба'): 5.0860743478968065,
 ('вынести', 'постановление'): 4.458188570801266,
 ('получить', 'доля'): -100,
 ('вынести', 'приговор'): 5.035955570118217,
 ('принять', 'решение'): 4.2708104493563575,
 ('признать', 'ответственность'): -100,
 ('выплатить', 'компенсация'): 8.029923912350435,
 ('обжаловать', 'санкция

In [29]:
do_likelihood_scores

{('подать', 'иск'): 100.13973739998514,
 ('признать', 'договор'): 1.5536002840528815,
 ('рассмотреть', 'вопрос'): 25.48100430402684,
 ('обвинить', 'руководство'): 10.518818474681158,
 ('получить', 'срок'): 7.901136881786522,
 ('обвинить', 'олигарх'): -100,
 ('направить', 'ведомство'): -100,
 ('просить', 'суд'): 32.43539509577039,
 ('делать', 'заявление'): 9.697346662833288,
 ('рассматривать', 'ходатайство'): -100,
 ('отказать', 'жалоба'): -100,
 ('делать', 'шаг'): -100,
 ('иметь', 'вид'): -100,
 ('оставить', 'заключение'): -100,
 ('дать', 'согласие'): 21.166493406202285,
 ('признать', 'сделка'): 2.3634722949507196,
 ('обвинить', 'господин'): 1.8970757799174138,
 ('удовлетворить', 'жалоба'): 36.27358456387836,
 ('вынести', 'постановление'): 8.603522494238348,
 ('получить', 'доля'): -100,
 ('вынести', 'приговор'): 30.64982485028807,
 ('принять', 'решение'): 66.2101580191843,
 ('признать', 'ответственность'): -100,
 ('выплатить', 'компенсация'): 56.03195870402436,
 ('обжаловать', 'санкция

3. Подготавливаем "золотой стандарт" коллокаций для этого корпуса: берем словосочетания, которые попадают в топ-100 по всем метрикам и пересекаем со словарем глагольной сочетаемости.

In [30]:
overlap = set(top_100_dice.keys()) & set(top_100_pmi.keys()) & set(top_100_likelik.keys())

In [31]:
overlap

{('арестовывать', 'подозреваемый'),
 ('арестовывать', 'счет'),
 ('взыскать', 'деньги'),
 ('вынести', 'вердикт'),
 ('выплатить', 'дивиденд'),
 ('выплатить', 'иркутянин'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('выплатить', 'штрафа'),
 ('дать', 'добро'),
 ('дать', 'облигация'),
 ('дать', 'подписка'),
 ('дать', 'показание'),
 ('дать', 'согласие'),
 ('делать', 'замечание'),
 ('доказать', 'бездействие'),
 ('доказать', 'вина'),
 ('доказать', 'невиновность'),
 ('доказать', 'неправомерность'),
 ('доказать', 'правность'),
 ('запретить', 'деятельность'),
 ('запретить', 'организацией|вчер'),
 ('заявить', 'отвод'),
 ('иметь', 'доверенность'),
 ('иметь', 'место'),
 ('использовать', 'водомет'),
 ('использовать', 'женщинсмертница'),
 ('направить', 'альтернативщик'),
 ('объявить', 'перерыв'),
 ('объявить', 'предприниматель'),
 ('обязать', 'горсовет'),
 ('оспаривать', 'отказ'),
 ('оспаривать', 'предписание'),
 ('оспорить', 'правомерность'),
 ('оспорить', 'предписание'),
 ('оспорить',

In [32]:
collocs = []
with open('verb_coll.txt', 'r', encoding = 'utf-8') as f:
    dict_verb = f.readlines()
for line in dict_verb:
    parts = line.strip().split('\t')
    colloc = parts[-1].split()
    if len(colloc) > 1:
        collocs.append((colloc[0], colloc[-1]))

In [33]:
collocs = list(set(collocs))

In [34]:
set(collocs) & set(overlap)

{('дать', 'добро'),
 ('дать', 'согласие'),
 ('объявить', 'перерыв'),
 ('отменить', 'постановление'),
 ('получить', 'статус'),
 ('предъявить', 'обвинение'),
 ('принять', 'резолюция'),
 ('рассмотреть', 'вопрос')}

4. Добавляем в "золотой стандарт" словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть).

Добавим слова, которые не вошли в словарь. Это юридические термины и выражения, которые имеют непосрдественное отношение к тексту. Думаю, что в ручной разметке их стоит выделить, так как они очень часто употребляются в текстах таких жанров.

In [35]:
new_collocs = [('выдавать', 'ордер'),
 ('выдавать', 'санкция'),
 ('выигрывать', 'суд|суд'),
 ('давать', 'показание'),
 ('добиваться', 'экстрадиция'),
 ('заявлять', 'отвод'),
 ('иметь', 'место'),
 ('направлять', 'запрос'),
 ('объявлять', 'голодовка'),
 ('объявлять', 'перерыв'),
 ('отменять', 'запрет'),
 ('подавать', 'апелляция'),
 ('представлять', 'угроза'),
 ('прекращать', 'поставка'),
 ('прекращать', 'производство'),
 ('признавать', 'правота'),
 ('проводить', 'обыск')]

In [36]:
gold_standard = list(set(collocs) & set(overlap)) + new_collocs

5. Оценить ранговую корреляцию (коэффициент Спирмена) результатов по каждой метрике с ЗС. Можно использовать scipy.stats.spearmanr. Описать ошибки каждой метрики.

In [38]:
spearmanr([' '.join(x) for x in top_100_dice.keys()][:len(gold_standard)], [' '.join(x) for x in gold_standard])

SpearmanrResult(correlation=0.27692307692307694, pvalue=0.1802089385790167)

In [39]:
spearmanr([' '.join(x) for x in top_100_pmi.keys()][:len(gold_standard)], [' '.join(x) for x in gold_standard])

SpearmanrResult(correlation=-0.3384615384615385, pvalue=0.09793932793665742)

In [40]:
spearmanr([' '.join(x) for x in top_100_likelik.keys()][:len(gold_standard)], [' '.join(x) for x in gold_standard])

SpearmanrResult(correlation=0.19153846153846155, pvalue=0.3590408645895138)

По всем трем метрикам получились неудачные результаты, наиболее значимые показатели у PMI.